In [1]:
import transformers
import torch
import pandas as pd
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, Trainer, TrainingArguments, BartForConditionalGeneration
from transformers.modeling_outputs import BaseModelOutput
from torch.optim import Adam
from accelerate import Accelerator
import wandb
from tqdm import tqdm
from torch.optim import AdamW

In [2]:
from evaluate import load
from rouge_score import rouge_scorer
from bert_score import score as bert_score

# meteor = load("meteor")
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [3]:
accelerator = Accelerator()

In [4]:
!wandb login --relogin --verify f59d448beb3315f3efbc5a0a80d9d2c346926308

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/dhruv/.netrc
wandb: Currently logged in as: noel22338 (nlp_project_team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
training_data = pd.read_csv('../Dataset/train.csv')
testing_data = pd.read_csv('../Dataset/test.csv')
validation_data = pd.read_csv('../Dataset/validation.csv')

In [6]:
print(training_data.head())

                                          hatespeech       csType  \
0  Maybe the UN could talk to those asian and afr...  Informative   
1  Maybe the UN could talk to those asian and afr...  Questioning   
2  Maybe the UN could talk to those asian and afr...   Denouncing   
3  Maybe the UN could talk to those asian and afr...     Positive   
4  Juice Jews are worse that nukes. Caption to a ...  Informative   

                                       counterspeech Suggest  Relevance  \
0  The us is the second most polluting country in...       3        4.0   
1  Doesn't everyone on the planet have a responsi...                3.0   
2  The world would be a better place if people we...       1        1.0   
3  You're right, ocean pollution is one of the ma...       3        4.0   
4  Anti-semitism is a serious problem that we nee...       2        3.0   

   Aggressive  Complexity  Comments source  \
0         2.0         3.0       NaN  Human   
1         2.0         2.0       NaN  Human

In [7]:
columns = training_data.columns
print(columns)

Index(['hatespeech', 'csType', 'counterspeech', 'Suggest', 'Relevance',
       'Aggressive', 'Complexity', 'Comments', 'source', 'claim',
       'centralTopic', 'speakerIntent', 'targetGroup', 'relevantPowerDynamics',
       'hatespeechImplication', 'targetGroupEmotionalReaction',
       'targetGroupCognitiveReaction', 'hatespeechOffensiveness', 'id',
       'is_high_quality', 'hs_id', 'hatespeechTarget', 'powerDynamics',
       'prompt_offensiveness', 'prompt_target_group', 'prompt_speaker_intent',
       'prompt_power_dynamics', 'prompt_implication',
       'prompt_emotional_reaction', 'prompt_cognitive_reaction',
       'prompt_cs_generation'],
      dtype='object')


In [8]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import torch

class DialoGPTDataset(Dataset):
    def __init__(self, data):
        self.data = data.reset_index(drop=True)
        self.tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
        self.bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

        # Intent label mapping
        self.categories = {
            'informative': 0,
            'questioning': 1,
            'denouncing': 2,
            'positive': 3,
            'humor': 4
        }

        # ✅ Create a mapping from hate speech → list of intent labels
        self.intent_map = (
            data.groupby("hatespeech")["csType"]
            .apply(lambda x: [self.categories[t.lower()] for t in x.unique()])
            .to_dict()
        )

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # Tokenize hate speech
        hate_inputs = self.tokenizer(
            row["hatespeech"],
            return_tensors='pt',
            max_length=128,
            truncation=True,
            padding="max_length"
        )

        # Tokenize counterspeech
        counter_inputs = self.bart_tokenizer(
            row["counterspeech"],
            return_tensors='pt',
            max_length=128,
            truncation=True,
            padding="max_length"
        )

        intent_id = torch.tensor(self.categories[row["csType"].lower()], dtype=torch.long)
        all_intents = self.intent_map[row["hatespeech"]]  # ✅ Look up all intents for this hate speech

        return {
            'input_ids': hate_inputs['input_ids'].squeeze(0),
            'attention_mask': hate_inputs['attention_mask'].squeeze(0),
            'counter_speech': counter_inputs['input_ids'].squeeze(0),
            'intent_id': intent_id,
            'raw_text': row["hatespeech"],
            'all_intents': all_intents  
        }

    def __len__(self):
        return len(self.data)


def custom_collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    counter_speech = torch.stack([item['counter_speech'] for item in batch])
    intent_id = torch.stack([item['intent_id'] for item in batch])
    all_intents = [item['all_intents'] for item in batch]
    raw_inputs = [item['raw_text'] for item in batch]

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'counter_speech': counter_speech,
        'intent_id': intent_id,
        'all_intents': all_intents,
        'raw_inputs':raw_inputs
    }



In [9]:
train_dataset = DialoGPTDataset(training_data)
test_dataset = DialoGPTDataset(testing_data)
validation_dataset = DialoGPTDataset(validation_data)

print(len(train_dataset))
print(len(test_dataset))
print(len(validation_dataset))

print(train_dataset[1])


9532
2971
1470
{'input_ids': tensor([  101,  2672,  1996,  4895,  2071,  2831,  2000,  2216,  4004,  1998,
         3060,  3741,  3625,  2005,  3938,  1009,  1997,  1996, 10796,  1999,
         1996, 17401,  2612,  1997, 22604,  2006,  2023, 14636,  2055,  4785,
         2689,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,

In [10]:
class FeatureEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, next_input):
        super(FeatureEncoder, self).__init__()
        self.model = AutoModel.from_pretrained('GroNLP/hateBERT')
        self.hidden_dim = hidden_dim
        self.output_size = next_input

        self.informative_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )

        self.questioning_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )

        self.denouncing_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )

        self.positive_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )

        self.humor_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )


    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        hate_speech_h = outputs.last_hidden_state[:, 0, :]

        informative_e = self.informative_head(hate_speech_h)
        questioning_e = self.questioning_head(hate_speech_h)
        denouncing_e = self.denouncing_head(hate_speech_h)
        positive_e = self.positive_head(hate_speech_h)
        humor_e = self.humor_head(hate_speech_h)

        return informative_e, questioning_e, denouncing_e, positive_e, humor_e, hate_speech_h

In [11]:
feature_encoder = FeatureEncoder(input_dim=128, hidden_dim=768, next_input=100)
print(feature_encoder(train_dataset[0]['input_ids'].unsqueeze(0), train_dataset[0]['attention_mask'].unsqueeze(0)))

(tensor([[0.0000, 0.0602, 0.0000, 0.2823, 0.4089, 0.0245, 0.1707, 0.0402, 0.2651,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6941, 0.3184, 0.2305, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0641, 0.0000, 0.0000, 0.0000,
         0.1692, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2387, 0.0000,
         0.0000, 0.0000, 0.1257, 0.1650, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0634, 0.1901, 0.0520, 0.0000, 0.0000, 0.0000, 0.0944, 0.0000, 0.1481,
         0.4632, 0.0000, 0.1444, 0.0000, 0.0000, 0.1218, 0.0571, 0.0000, 0.0000,
         0.2463, 0.0000, 0.1078, 0.0000, 0.0000, 0.0000, 0.0769, 0.0000, 0.0000,
         0.0000, 0.0000, 0.3521, 0.0000, 0.2064, 0.0000, 0.4716, 0.0000, 0.0000,
         0.5140, 0.0000, 0.0000, 0.8616, 0.0698, 0.0000, 0.7839, 0.0000, 0.7415,
         0.2588, 0.0000, 0.0000, 0.0552, 0.0000, 0.0000, 0.0174, 0.0000, 0.0000,
         0.0000]], grad_fn=<ReluBackward0>), tensor([[0.0915, 0.0609, 0.0000, 0.0912, 0.0000, 0.0000, 0.3167

In [12]:
class CounterSpeechNetwork(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, encoder_output, max_length):
        super(CounterSpeechNetwork, self).__init__()

        self.feature_encoder = FeatureEncoder(input_dim, hidden_dim, encoder_output)

        self.informative_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
        self.questioning_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
        self.denouncing_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
        self.positive_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
        self.humor_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

        self.informative_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.informative_decoder.config.d_model)
        self.questioning_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.questioning_decoder.config.d_model)
        self.denouncing_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.denouncing_decoder.config.d_model)
        self.positive_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.positive_decoder.config.d_model)
        self.humor_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.humor_decoder.config.d_model)

        self.tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
        self.max_length = max_length

    def forward(self, input_ids, attention_mask, intent_id, counter_speech=None):
        informative_e, questioning_e, denouncing_e, positive_e, humor_e, hate_speech_h = self.feature_encoder(input_ids, attention_mask)

        batch_size = input_ids.size(0)

        fused = torch.zeros(batch_size, 1, self.informative_decoder.config.d_model, device=input_ids.device)
    
        for i in range(batch_size):
            if intent_id[i] == 0:
                fused[i] = self.informative_fusion(torch.cat((hate_speech_h[i], informative_e[i]), dim=-1)).unsqueeze(0)
            elif intent_id[i] == 1:
                fused[i] = self.questioning_fusion(torch.cat((hate_speech_h[i], questioning_e[i]), dim=-1)).unsqueeze(0)
            elif intent_id[i] == 2:
                fused[i] = self.denouncing_fusion(torch.cat((hate_speech_h[i], denouncing_e[i]), dim=-1)).unsqueeze(0)
            elif intent_id[i] == 3:
                fused[i] = self.positive_fusion(torch.cat((hate_speech_h[i], positive_e[i]), dim=-1)).unsqueeze(0)
            elif intent_id[i] == 4:
                fused[i] = self.humor_fusion(torch.cat((hate_speech_h[i], humor_e[i]), dim=-1)).unsqueeze(0)
            else:
                raise ValueError(f"Invalid intent_id: {intent_id[i]}")

        if counter_speech is not None:
            losses = []
            for i in range(batch_size):
                if intent_id[i] == 0:
                    output = self.informative_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                elif intent_id[i] == 1:
                    output = self.questioning_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                elif intent_id[i] == 2:
                    output = self.denouncing_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                elif intent_id[i] == 3:
                    output = self.positive_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                elif intent_id[i] == 4:
                    output = self.humor_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                losses.append(output.loss)
            avg_loss = sum(losses) / len(losses)  # Average loss across the batch
            return None, avg_loss  # No decoded text during training
        else:
            decoded_texts = []
            for i in range(batch_size):
                if intent_id[i] == 0:
                    output = self.informative_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                elif intent_id[i] == 1:
                    output = self.questioning_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                elif intent_id[i] == 2:
                    output = self.denouncing_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                elif intent_id[i] == 3:
                    output = self.positive_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                elif intent_id[i] == 4:
                    output = self.humor_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                decoded_texts.append(self.tokenizer.decode(output[0], skip_special_tokens=True))
            return decoded_texts, None  # Decoded text during inference, no loss
    def judge_responses(self,input_ids, attention_mask,counter_speech=None):
        
        informative_e, questioning_e, denouncing_e, positive_e, humor_e, hate_speech_h = self.feature_encoder(input_ids, attention_mask)

        batch_size = input_ids.size(0)
        output_dict = {'informative':[],'questioning':[],'denouncing':[],'positive':[],'humor':[]}
        informative_fused = torch.zeros(batch_size, 1, self.informative_decoder.config.d_model, device=input_ids.device)
        questioning_fused = torch.zeros(batch_size, 1, self.informative_decoder.config.d_model, device=input_ids.device)
        denouncing_fused = torch.zeros(batch_size, 1, self.informative_decoder.config.d_model, device=input_ids.device)
        positive_fused = torch.zeros(batch_size, 1, self.informative_decoder.config.d_model, device=input_ids.device)
        humor_fused = torch.zeros(batch_size, 1, self.informative_decoder.config.d_model, device=input_ids.device)
        
    
        for i in range(batch_size):
            informative_fused[i] = self.informative_fusion(torch.cat((hate_speech_h[i], informative_e[i]), dim=-1)).unsqueeze(0)
            questioning_fused[i] = self.questioning_fusion(torch.cat((hate_speech_h[i], questioning_e[i]), dim=-1)).unsqueeze(0)
            denouncing_fused[i] = self.denouncing_fusion(torch.cat((hate_speech_h[i], denouncing_e[i]), dim=-1)).unsqueeze(0)
            positive_fused[i] = self.positive_fusion(torch.cat((hate_speech_h[i], positive_e[i]), dim=-1)).unsqueeze(0)
            humor_fused[i] = self.humor_fusion(torch.cat((hate_speech_h[i], humor_e[i]), dim=-1)).unsqueeze(0)


            output_dict['informative'].append(self.informative_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=informative_fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0)))
            output_dict['questioning'].append(self.questioning_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=questioning_fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0)))
            output_dict['denouncing'].append(self.denouncing_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=denouncing_fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0)))
            output_dict['positive'].append(self.positive_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=positive_fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0)))
            output_dict['humor'].append(self.humor_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=humor_fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0)))


        return output_dict
            


        


In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)

counterspeechnetwork = CounterSpeechNetwork(input_dim=128, hidden_dim=768, encoder_output=256, max_length=50)

for batch in train_dataloader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    intent_id = batch['intent_id']
    counter_speech = batch['counter_speech']

    print(counterspeechnetwork(input_ids, attention_mask, intent_id, counter_speech))
    break

In [ ]:
model = CounterSpeechNetwork(input_dim=128, hidden_dim=768, encoder_output=256, max_length=50)
optimizer = AdamW(model.parameters(), lr=5e-5)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True,collate_fn = custom_collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True,collate_fn = custom_collate_fn)
validation_dataloader = DataLoader(validation_dataset, batch_size=32, shuffle=True,collate_fn = custom_collate_fn)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 10

for epoch in range(epochs):
    model.train()
    total_train_loss = 0.0

    # Wrap train_dataloader with tqdm for training progress
    train_loop = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs} [Train]", leave=False)
    for batch in train_loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        counter_speech = batch['counter_speech'].to(device)
        intent_ids = batch['intent_id'].to(device)

        optimizer.zero_grad()

        # Process the entire batch at once
        _, loss = model(input_ids, attention_mask, intent_ids, counter_speech)

        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

        # Update tqdm with current batch loss
        train_loop.set_postfix({'batch_loss': loss.item(), 'avg_loss': total_train_loss / (train_loop.n + 1)})

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss:.4f}")
    
    # Validation
    model.eval()
    total_val_loss = 0.0

    # Wrap validation_dataloader with tqdm for validation progress
    val_loop = tqdm(validation_dataloader, desc=f"Epoch {epoch + 1}/{epochs} [Validation]", leave=False)
    with torch.no_grad():
        for batch in val_loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            counter_speech = batch['counter_speech'].to(device)
            intent_ids = batch['intent_id'].to(device)

            # Process the entire batch at once
            _, loss = model(input_ids, attention_mask, intent_ids, counter_speech)

            total_val_loss += loss.item()

            # Update tqdm with current batch loss
            val_loop.set_postfix({'batch_loss': loss.item(), 'avg_loss': total_val_loss / (val_loop.n + 1)})

    avg_val_loss = total_val_loss / len(validation_dataloader)
    print(f"Epoch {epoch + 1}/{epochs} | Validation Loss: {avg_val_loss:.4f}")

In [ ]:
torch.save(model.state_dict(), 'architecture_parameters.pth')

In [ ]:
# Test evaluation
model.eval()
test_predictions = []
test_references = []

test_loop = tqdm(test_dataloader, desc="Test Evaluation", leave=True)
with torch.no_grad():
    for batch in test_loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        counter_speech = batch['counter_speech'].to(device)  # Reference texts
        intent_ids = batch['intent_id'].to(device)

        predictions, _ = model(input_ids, attention_mask, intent_ids)

        references = [model.tokenizer.decode(cs, skip_special_tokens=True) for cs in counter_speech]

        test_predictions.extend(predictions)
        test_references.extend(references)

        test_loop.set_postfix({'predictions': len(test_predictions)})

# Final output
print(f"Generated {len(test_predictions)} test predictions")

In [ ]:
P, R, F1 = bert_score(test_predictions, test_references, lang="en", verbose=True)

# Final output
print(f"Generated {len(test_predictions)} test predictions")
print(f"BERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

In [ ]:
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
for pred, ref in zip(test_predictions, test_references):
    scores = scorer.score(ref, pred)  # ROUGE scores for each prediction-reference pair
    rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
    rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
    rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

# Compute average ROUGE scores
avg_rouge1 = sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1'])
avg_rouge2 = sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2'])
avg_rougeL = sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL'])

print(f"ROUGE - Rouge1: {avg_rouge1:.4f}, Rouge2: {avg_rouge2:.4f}, RougeL: {avg_rougeL:.4f}")

In [ ]:
for pred, ref in zip(test_predictions, test_references):
    print(f'Generated: {pred}')
    print(f'Actual: {ref}')
    print()

In [13]:
model = CounterSpeechNetwork(input_dim=128, hidden_dim=768, encoder_output=256, max_length=50)
model.load_state_dict(torch.load("architecture_parameters.pth"))
model.eval()

/tmp/ipykernel_1406414/785172572.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("architecture_parameters.pth"))


CounterSpeechNetwork(
  (feature_encoder): FeatureEncoder(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_feature

In [14]:
def judge_scoring(counterspeeches, device="cuda"):

    judge_tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
    judge_model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small").to(device)
    judge_model.eval()

    selected_intents = {}

    for hate_speech, intent_responses in counterspeeches.items():
        best_intent = None
        best_score = float("-inf")

        for intent, response in intent_responses.items():
            # Construct input for DialoGPT: HateSpeech + EOS + Response
            input_text = f"{hate_speech} {judge_tokenizer.eos_token} {response}"
            input_ids = judge_tokenizer.encode(input_text, return_tensors="pt").to(device)

            with torch.no_grad():
                output = judge_model(input_ids, labels=input_ids)
                score = -output.loss.item()  # Higher is better (negative loss)

            if score > best_score:
                best_score = score
                best_intent = intent

        selected_intents[hate_speech] = best_intent

    return selected_intents


In [15]:
!huggingface-cli login

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/home/dhruv/miniforge3/envs/

In [18]:
def evaluate_with_judge_model(model, test_dataloader, device):
    categories = {
        'informative': 0,
        'questioning': 1,
        'denouncing': 2,
        'positive': 3,
        'humor': 4
    }

    model.to(device)
    model.eval()

    from transformers import AutoTokenizer, AutoModelForCausalLM
    judge_tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
    judge_model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small").to(device)
    judge_model.eval()
    print("Judge model ready!")

    total_samples = 0
    correct_predictions = 0

    test_loop = tqdm(test_dataloader, desc="Test Evaluation", leave=True)
    with torch.no_grad():
        for batch in test_loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            counter_speech = batch['counter_speech'].to(device)
            raw_inputs = batch['raw_inputs']
            all_intents = batch['all_intents']  # list of gold intents per sample (as int indices)

            outputs = model.judge_responses(input_ids, attention_mask, counter_speech)

            for i in range(len(input_ids)):
                intent_scores = {}

                for intent in ["informative", "questioning", "denouncing", "positive", "humor"]:
                    response = model.tokenizer.decode(
                        torch.argmax(outputs[intent][i].logits, dim=-1)[0],
                        skip_special_tokens=True
                    )

                    # Prompting for a rating
                    prompt = (
                        f"Hate speech: {raw_inputs[i]}\n"
                        f"Proposed counterspeech (intent: {intent}): {response}\n\n"
                        f"On a scale of 1 to 10, how appropriate and effective is this counterspeech in response to the hate speech? Just respond with a number."
                    )

                    judge_input = judge_tokenizer.encode(prompt, return_tensors='pt').to(device)
                    output_ids = judge_model.generate(judge_input, max_new_tokens=10, pad_token_id=judge_tokenizer.eos_token_id)
                    score_text = judge_tokenizer.decode(output_ids[0], skip_special_tokens=True)

                    try:
                        # Extract the first number in response (robust to "I would give it a 7" etc.)
                        score = next((float(s) for s in score_text.split() if s.replace('.', '', 1).isdigit()), 0)
                        score = max(0, min(score, 10))  # Clamp between 0 and 10
                    except:
                        score = 0  # Fallback score if judge LM fails

                    intent_scores[intent] = score

                # Pick best scoring intent
                best_intent = max(intent_scores, key=intent_scores.get)
                best_intent_idx = categories[best_intent]

                if best_intent_idx in all_intents[i]:
                    correct_predictions += 1
                total_samples += 1

            test_loop.set_postfix({'accuracy': correct_predictions / total_samples if total_samples else 0})

    final_accuracy = correct_predictions / total_samples if total_samples else 0
    print(f"\nFinal Category Accuracy using Judge LM (Rating 1–10): {final_accuracy:.4f}")
    return final_accuracy


In [20]:

test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True,collate_fn = custom_collate_fn)
evaluate_with_judge_model(model,test_dataloader,device = torch.device("cuda"))

Judge model ready!


Test Evaluation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 93/93 [06:42<00:00,  4.33s/it, accuracy=0.736]


Final Category Accuracy using Judge LM (Rating 1–10): 0.7358


0.7357791989229215

In [ ]:
!nvidia-smi